In [187]:
import pandas as pd
import os

# Define the directory and file names
directory = os.getcwd() + "/data/"

# Read the parquet files
interventions_bxl = pd.read_parquet(directory + "interventions_bxl.parquet.gzip")
interventions_bxl2 = pd.read_parquet(directory + "interventions_bxl2.parquet.gzip")
interventions1 = pd.read_parquet(directory + "interventions1.parquet.gzip")
interventions2 = pd.read_parquet(directory + "interventions2.parquet.gzip")
interventions3 = pd.read_parquet(directory + "interventions3.parquet.gzip")
cad9 = pd.read_parquet(directory + "cad9.parquet.gzip")
aed_locations = pd.read_parquet(directory + "aed_locations.parquet.gzip")
ambulance_locations = pd.read_parquet(directory + "ambulance_locations.parquet.gzip")
mug_locations = pd.read_parquet(directory + "mug_locations.parquet.gzip")
pit_locations = pd.read_parquet(directory + "pit_locations.parquet.gzip")

In [52]:
print(len(aed_locations))

15227


In [53]:
# we want to store all AED's considered poorly performing and the reason for why they are considered so. During the data cleaning, 
# some AED's will already get filtered, and are considered poorly performing.

# creating a dataframe to store these AED's. Only the ID and the reason for removal are stored.
removed_AED = pd.DataFrame(columns = ['id','reason'])

# function to filter and store the AED's
def filter_and_log_removed(df, filter_condition, reason):
    # apply the filter condition
    removed = df[filter_condition]
    
    # append removed devices to the removed_AED DataFrame
    global removed_AED
    removed_AED = pd.concat([removed_AED, pd.DataFrame({'id': removed['id'], 'reason': reason})])
    
    # Remove the filtered devices from the original DataFrame
    df = df.drop(removed.index)
    
    return df

In [54]:
# count the number of missing addresses in aed_locations
aed_missing_location_count = aed_locations['address'].isna().sum()
print(aed_missing_location_count)

# filter out instances where address is missing. Those instances are invalid, since we cannot know where the AED was placed
# first we inspect these instances, perhaps address was present under the wrong variable
aed_locations_with_na = aed_locations[aed_locations['address'].isna()]
print(aed_locations_with_na)

# apply our function in order to filter out and store the filtered instances
aed_locations_cleaned = filter_and_log_removed(aed_locations, aed_locations['address'].isna(), 'address_missing')

2
          id  type address  number  postal_code  municipality    province  \
6092  7323.0  None    None     NaN       3910.0      Neerpelt     Limburg   
6212  7467.0  None    None     0.0       6870.0  Saint-Hubert  Luxembourg   

     location public available hours  
6092     None   None      None  None  
6212     None   None      None  None  


C:\Users\eloua\AppData\Local\Temp\ipykernel_26412\612258699.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  removed_AED = pd.concat([removed_AED, pd.DataFrame({'id': removed['id'], 'reason': reason})])


In [55]:
len(aed_locations_cleaned) #check

15225

In [56]:
aed_missing_number_count = aed_locations['number'].isna().sum()
print(aed_missing_number_count)

# proportion missing:
print(aed_missing_number_count/aed_locations_cleaned.shape[0])

# around 14% of the cleaned instances has a missing streetnumber. This is a significant amount, which means that simply removing them would have too
#much of an impact on the results. Therefore an alternative solution needs to be found.(midden van de straat, maar dat is wel vrij intensief? hoe weet
#je wat het straatnummer is in het midden van de straat?)
#in a real world scenario, one might ask the stakeholder what happened with this data, perhaps the data is available somewhere else.

2142
0.1406896551724138


In [57]:
aed_type_counts = aed_locations['type'].value_counts()
print(aed_type_counts)

aed_missing_type_count = aed_locations['type'].isna().sum()
print(aed_missing_type_count)
print(len(aed_locations))
# the majority of the known AED types in the cleaned table is fixed apparatus (5166, or 33.93%) , with mobile apparatus only a very small part 
#(17, or 0.11%) of the group, and only #one of M5066A type. The majority is of unknown type (2142, or 66.07%).
#in a real world scenario it would again be possible to ask the stakeholder about these missing values.
#we assume that all types of devices are useable.

type
Appareil fixe-Vast apparaat         5048
Appareil fixe                         66
Appareil Fixe                         19
Appareil Mobile- Mobiel apparaat      17
Vast apparaat                         16
M5066A                                 1
Name: count, dtype: int64
10060
15227


In [58]:
#we can remove the type column, we don't need it for further analysis
aed_locations_cleaned = aed_locations_cleaned.drop(columns = ['type'])

In [59]:
aed_locations.head(10)

,id,type,address,number,postal_code,municipality,province,location,public,available,hours
0,13.0,None,Blvd. Fr. Roosevelt,24.0,7060.0,SOIGNIES,Hainaut,None,Y,None,None
1,70.0,None,Ch. De Wégimont,76.0,4630.0,Ayeneux,Liège,None,None,None,None
2,71.0,None,Place Saint - Lambert,NaN,4020.0,Liège,Liège,None,None,None,None
3,72.0,None,Rue du Doyard,NaN,4990.0,Lierneux,Liège,None,None,None,None
4,73.0,None,Fond Saint Servais,NaN,4000.0,Liège,Liège,None,None,None,None
5,74.0,None,Rue des Prémontrès,12.0,4020.0,Liège,Liège,None,None,None,None
6,75.0,None,Route de Bastogne,1.0,4920.0,Harzé,Liège,None,None,None,None
7,76.0,None,Rue du Parc,1.0,4540.0,Jehay,Liège,None,None,None,None
8,77.0,None,Blvd. De la Constitution,19.0,4020.0,Liège,Liège,None,None,None,None
9,78.0,None,Place de la République française,NaN,4000.0,Liège,Liège,None,None,None,None


In [60]:
aed_location_counts = aed_locations['location'].value_counts()
print(aed_location_counts)
#the location variable contains a description of where the AED is located exactly. This is not useable for the analysis, so the variable is simply
#removed later on
#aed_locations_cleaned.drop(columns = ['location'])

location
/                                              314
.                                              289
Accueil                                        196
                                                89
INKOM                                           81
                                              ... 
Gelijkvloers buiten tegen loods                  1
Hoofdgebouw, 1ste verdieping, thv cafetaria      1
reddersgebouw aan zwemvijver                     1
Locatie : ingang poort hoofdgebouw               1
 inkomhal, glvl                                  1
Name: count, Length: 5732, dtype: int64


In [61]:
aed_public_counts = aed_locations['public'].value_counts()
print(aed_public_counts)
print(aed_locations['public'].isna().sum())
#1220 devices are not publicly available. It would be useful to have more information on these, to decide whether or not they could still be used in
#case of a cardiac arrest.
aed_nonpublic_aed = aed_locations[(aed_locations['public'] == 'Non-Nee') | (aed_locations['public'] == 'N')]
aed_nonpublic_aed_with_available = aed_nonpublic_aed.dropna(subset=['available'])#only check for the ones that have a value for available, then we can
#decide for those if they are useable
print(aed_nonpublic_aed_with_available['available'].value_counts())
#there seem to be contradictions with the data, where it says for the public variable that it is not available, but for the available variable that it is
#available. Only a very small amount actually gives more details.
#remove degene die op beide nee zeggen? en ook diegene die enkel op bepaalde uren beschikbaar zijn? in principe zou je ze nog altijd kunnen gebruiken,
#je zou ze kunnen markeren, maar dat is zoveel moeite, zoveel aanpassingen aan het project, voor slechts 15 aeds ofzo. kijk ook eens naar de counts voor
#de andere, die wel 'ja' publically available zijn.
#we are optimistic, and asume that if there is one yes in either the public variable or the available variable, we assume that it is a useable AED.
#there are 14 other AED's which give the hours at which they are available. These are available most of the time, so they are considered to be 
#available.

public
Oui-Ja     3937
Y          2775
Non-Nee    1109
y           129
N           111
Oui          11
Ja            6
J             1
Name: count, dtype: int64
7148
available
Non-Nee                                                          748
Oui-Ja                                                           230
Pendant heures d ouverture du site                                 1
Nee                                                                1
8:00 - 17:00                                                       1
Heure de bureau en semaine                                         1
Heures de bureau                                                   1
Heures de bureau                                                   1
selon heures d ouverture d Euro-Délices                            1
enkel tijdens de kantooruren (8 - 19u)                             1
tijdens werkuren                                                   1
Accessible par toute personne présente dans l inrfastructure.    

In [62]:
#removing the AED's for which both the available and the public variables contain a negative
# the conditions
filter_condition = ((aed_locations['public'] == 'Non-Nee') | (aed_locations['public'] == 'N')) & ((aed_locations['available'] == 'Non-Nee') |
                                                                                                 (aed_locations['available'] == 'Nee'))
#there was a single entry for which the value of the variable available was simply "nee" instead of "Non-Nee"
aed_locations_cleaned = filter_and_log_removed(aed_locations_cleaned, filter_condition, 'not_available')

#we can now remove the available and the public variables, since we do not need them in the further analysis
aed_locations_cleaned = aed_locations_cleaned.drop(columns = ['available', 'public', 'location'])

C:\Users\eloua\AppData\Local\Temp\ipykernel_26412\612258699.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  removed = df[filter_condition]


In [63]:
#checking the value of hours for the remaining entries
aed_hours_counts = aed_locations['hours'].value_counts()
print(aed_hours_counts)
print(aed_locations['hours'].isna().sum())
print(aed_locations_cleaned['hours'].isna().sum())
print(1 - aed_locations_cleaned['hours'].isna().sum()/len(aed_locations_cleaned))
#most values for the hours seem to overlap with office hours. only 5.42% of the instances has a value for hours. We consider the AED's available at
#office hours to be available when they are needed.

hours
8h-17h                                                                                                                            42
8h-16h                                                                                                                            20
24/24                                                                                                                             19
HEURE DE BUREAU                                                                                                                   13
Tijdens openingsuren recyclagepark                                                                                                10
                                                                                                                                  ..
ma-vrij 8u-17u                                                                                                                     1
En fonction des locations                                      

In [64]:
#we can remove the hours variable now
aed_locations_cleaned.drop(columns = ['hours'])

,id,address,number,postal_code,municipality,province
0,13.0,Blvd. Fr. Roosevelt,24.0,7060.0,SOIGNIES,Hainaut
1,70.0,Ch. De Wégimont,76.0,4630.0,Ayeneux,Liège
2,71.0,Place Saint - Lambert,NaN,4020.0,Liège,Liège
3,72.0,Rue du Doyard,NaN,4990.0,Lierneux,Liège
4,73.0,Fond Saint Servais,NaN,4000.0,Liège,Liège
...,...,...,...,...,...,...
15220,16660.0,Chaussée de Marche,799.0,5100.0,Wierde,Namur
15221,16661.0,Nekkerspoel-Borcht,19.0,2800.0,Mechelen,Antwerpen
15223,16664.0,Nieuwe Dreef,17.0,9160.0,Lokeren,Oost-Vlaanderen
15224,16665.0,Panterschipstraat,207.0,9000.0,Gent,Oost-Vlaanderen


In [66]:
#751 AED's were removed from the analysis. 2 because their address was missing, 749 because they were not available. The ones 
#where address is missing do not necessarily need to be reallocated, the ones that are not available could be allocated. Perhaps the AED's not available
#to the public are placed in large facilities, e.g. sports facilities, where they actually do serve a purpose. This would need to be verified for each 
#of these AEDs individually.
removed_AED

,id,reason
6092,7323.0,address_missing
6212,7467.0,address_missing
750,1412.0,not_available
1123,1881.0,not_available
3050,4024.0,not_available
...,...,...
15211,16650.0,not_available
15212,13572.0,not_available
15213,16651.0,not_available
15222,16662.0,not_available


In [ ]:
#for the other tables, we simply need to remove unnecessary variables, and merge the different tables.
#all the tables containing info can be merged into one big table, all the tables containing information from the first responders can be merged.

#de interventies tabellen bevatten ook info over de permanence. Die kan ook meegenomen worden, hoewel in principe die info al aanwezig is in de 
#locations tabellen.

In [200]:
#first we verify whether the unit is permanently staffed. If it is not, we do not consider it useable because it is considered
#unreliable, therefore an AED close to it's location can still be useful.
ambulance_value_counts = ambulance_locations['occasional_permanence'].value_counts()
print(ambulance_value_counts)
ambulance_locations_filtered = ambulance_locations[ambulance_locations['occasional_permanence'] == "N"]
occasional_permanence_ambulance = ambulance_locations[ambulance_locations['occasional_permanence'] == "Y"]



occasional_permanence
N    251
Y     28
Name: count, dtype: int64


In [130]:
#get the latitude and longitude of the PIT locations, since these are missing from the table.

#might not be 100% accurate

#some of the hospitals did not get recognized, so the address was manually looked up and inputted in the list. That way, the
#exact longitude and latitude could still be calculated.



# initialize geolocator
geolocator = Nominatim(user_agent="pit_locator")

# data in lists
# need to input the names manually in the list, since the useful part (for example "AZ RIVIERENLAND") is not consistently present in any variable.
#simply inputting the entire string value of the campus variable doesn't work, for example "104 - AZ RIVIERENLAND --- Campus/Site: 1270 - RUMST" yields
#no results.
data = {
    'name': ['AZ RIVIERENLAND ', 'AZ SINT-VINCENTIUS DEINZE', 'ZNA STUIVENBERG', ' HEILIG HART ZIEKENHUIS 1260',
             'HEILIG HART ZIEKENHUIS MOL', 'VITAZ CAMPUS LOKEREN', 'OLV ZIEKENHUIS ASSE', 'AZ SINT-JAN BRUGGE - OOSTENDE',
             'AZ SINT-LUCAS', 'AZ ZENO BLANKENBERGE', 'OOST-LIMBURG SINT-JAN GENK', 'UZ LEUVEN GASTHUISBERG',
             'HOPITAL IRIS SUD', 'CHU SAINT-PIERRE - PORTE DE HAL', 'UZ BRUSSEL', 'GRAND HOPITAL DE CHARLEROI SAINT- JOSEPH',
             'C.H.U. AMBROISE PARE', 'CHR SAMBRE ET MEUSE (SITE MEUSE)', 'CLINIQUE ANDRE RENARD', 'HOPITAL DE LA CITADELLE',
             'CHR VERVIERS - LA TOURELLE', None, None, None],
    'address': [None, None, None, None, None, None, None, None, None, None,
                None, 'Rue Jean Paquot 63', 'Boulevard de Waterloo 129' , '290, rue Haute', 'Rue Marguerite Depasse 6', 'Boulevard Président Kennedy 2',
                'Avenue Albert 1er 185', 'rue André Renard 1', None, None,
                None, 'Rue des Déportés 137', "Rue d'Harnoncourt 48", 'Avenue de Houffalize 35']
}
# create a DataFrame
hospitals_df = pd.DataFrame(data)

# DataFrame to store pit hospital coordinates
hospitals_with_coords = pd.DataFrame(columns=['id', 'name', 'municipality', 'address', 'latitude', 'longitude'])


# function to geocode hospitals, can use the names of the hospitals or the addresses if the name is not enough 
def geocode_hospitals(df):
    for index, row in df.iterrows():
        hospital_name = row['name']
        address = row['address']
        
        # determine the query based on available information
        if address:
            query = address
        else:
            query = f"{hospital_name}, Belgium"
        
        try:
            location = geolocator.geocode(query, timeout=10)
            if location:
                df.loc[index, 'address'] = location.address
                df.loc[index, 'latitude'] = location.latitude
                df.loc[index, 'longitude'] = location.longitude
            else:
                df.loc[index, 'address'] = None
                df.loc[index, 'latitude'] = None
                df.loc[index, 'longitude'] = None
        except (GeocoderTimedOut, GeocoderServiceError) as e:
            print(f"Error geocoding {query}: {e}")
            df.loc[index, 'address'] = None
            df.loc[index, 'latitude'] = None
            df.loc[index, 'longitude'] = None
    
    return df

# geocode the hospitals
pit_hospitals_with_coords = geocode_hospitals(hospitals_df)

#drop columns, we only need the coordinates
pit_hospitals_with_coords = pit_hospitals_with_coords.loc[:, ['latitude', 'longitude']]

# display the hospitals DataFrame with addresses and coordinates
print(pit_hospitals_with_coords)

     latitude  longitude
0   51.099783   4.232622
1   50.986425   3.527572
2   51.223649   4.434875
3   50.810888   4.933485
4   51.187526   5.114405
5   51.101340   4.000929
6   50.909904   4.196023
7   51.222791   2.914147
8   51.062547   3.720756
9   51.307809   3.124371
10  50.957775   5.517608
11  50.825930   4.378332
12  50.833473   4.345943
13  50.835695   4.348181
14  50.418054   4.480988
15  43.219179   0.079410
16  50.467587   4.888018
17  50.599613   5.631759
18  50.674721   5.633436
19  50.652037   5.578166
20  50.582211   5.859296
21  49.584723   3.054645
22  49.552645   5.526049
23  49.927931   5.384295


In [120]:
# now get and store the mug hospital coordinates
mug_hospitals_with_coords = pd.DataFrame(columns=['latitude', 'longitude'])

def geocode_hospitals(df):
    for index, row in df.iterrows():
        address = row['address_campus']
        municipality = row['municipality']
        
        # construct the query using address and municipality
        query = f"{address}, {municipality}, Belgium"
        
        try:
            location = geolocator.geocode(query, timeout=10)
            if location:
                df.loc[index, 'latitude'] = location.latitude
                df.loc[index, 'longitude'] = location.longitude
            else:
                df.loc[index, 'latitude'] = None
                df.loc[index, 'longitude'] = None
        except (GeocoderTimedOut, GeocoderServiceError) as e:
            print(f"Error geocoding {query}: {e}")
            df.loc[index, 'latitude'] = None
            df.loc[index, 'longitude'] = None

    return df

# geocode the hospitals
mug_hospitals_with_coords = geocode_hospitals(mug_locations)

# display the hospitals DataFrame with addresses and coordinates
print(mug_hospitals_with_coords)


    hospital_id  mug_id  campus_id                      name_hospital  \
0             9  102000       6230  ZIEKENHUISNETWERK ANTWERPEN (ZNA)   
1            99  102000       2020                  GZA- ZIEKENHUIZEN   
2           682  102000       1210                          AZ MONICA   
3             9  103000       2000  ZIEKENHUISNETWERK ANTWERPEN (ZNA)   
4            99  103000       1290                  GZA- ZIEKENHUIZEN   
..          ...     ...        ...                                ...   
89           20  808000       3010        CHR VERVIERS - EAST BELGIUM   
90          168  901000       3720                           VIVALIA    
91          246  902000       3690                           VIVALIA    
92          168  903000       3240                           VIVALIA    
93          164  904000       3230                           VIVALIA    

             name_campus           address_campus  postal_code  \
0              ZNA CADIX         KEMPENSTRAAT 100        

In [122]:
#remove all but the coordinate variables
mug_hospitals_with_coords = mug_hospitals_with_coords.loc[:, ['latitude', 'longitude']]
print(mug_hospitals_with_coords)

     latitude  longitude
0   51.231422   4.416284
1   51.205856   4.412845
2   51.206556   4.470778
3   51.401310   4.763188
4   51.174354   4.420085
..        ...        ...
89  50.715566   6.007128
90  50.159132   5.683118
91  49.678031   5.820700
92  49.927931   5.384295
93  50.228223   5.323121

[94 rows x 2 columns]


In [131]:
# put it into one big table, that contains all the coordinates
ambulance_locations_with_coords = ambulance_locations.loc[:, ['latitude', 'longitude']]
all_hospital_locations = pd.concat([mug_hospitals_with_coords, pit_hospitals_with_coords, ambulance_locations_with_coords], ignore_index = True)
all_hospital_locations

,latitude,longitude
0,51.231422,4.416284
1,51.205856,4.412845
2,51.206556,4.470778
3,51.401310,4.763188
4,51.174354,4.420085
...,...,...
392,50.025633,5.359641
393,50.674803,5.633665
394,50.652888,5.578356
395,50.622523,5.636381


In [ ]:
#we can simply put all the coordinates in one table, to get a single table of hospital coordinates.



In [188]:
# the coordinates of the hospitals should match the coordinates from the calls. dus dat kan ik verifiëren.
# if the data is good, there should be a perfect overlap between the hospital permanence and the interventions permanence coordinates. This will be
#verified.

interventions_bxl_coords = interventions_bxl.loc[:, ['latitude_permanence', 'longitude_permanence']]
interventions_bxl2_coords = interventions_bxl2.loc[:, ['Latitude Permanence', 'Longitude Permanence']]
interventions1_coords = interventions1.loc[:, ['Latitude permanence', 'Longitude permanence']]
interventions2_coords = interventions2.loc[:, ['Latitude permanence', 'Longitude permanence']]
interventions3_coords = interventions3.loc[:, ['Latitude permanence', 'Longitude permanence']]
cad9_coords = cad9.loc[:, ['Latitude permanence', 'Longitude permanence']]

# need to divide, there was no decimal point in the coordinates for the BXL interventions
# the inputted coordinates are not consistent in the amount of precision, so simply dividing all by 10**5 would not work, for example one coordinate 
#would be 500.123456, and another would be 5.00123456, while we want 50.0123456
#interventions_bxl_coords = interventions_bxl_coords/10**5
#interventions_bxl2_coords = interventions_bxl2_coords/10**5



# rename columns to match between the different datasets. The column names from interventions_bxl are used as the standard.
interventions_bxl2_coords = interventions_bxl2_coords.rename(columns={
    'Latitude Permanence': 'latitude_permanence',
    'Longitude Permanence': 'longitude_permanence'
})
interventions1_coords = interventions1_coords.rename(columns={
    'Latitude permanence': 'latitude_permanence',
    'Longitude permanence': 'longitude_permanence'
})
interventions2_coords = interventions2_coords.rename(columns={
    'Latitude permanence': 'latitude_permanence',
    'Longitude permanence': 'longitude_permanence'
})
interventions3_coords = interventions3_coords.rename(columns={
    'Latitude permanence': 'latitude_permanence',
    'Longitude permanence': 'longitude_permanence'
})
cad9_coords = cad9_coords.rename(columns={
    'Latitude permanence': 'latitude_permanence',
    'Longitude permanence': 'longitude_permanence'
})

In [189]:
#for the interventions_bxl and interventions_bxl2 tables the coordinates were not given with their decimal points, so we need to
#convert those coordinate values.
#finding the minimum value, so we can divide all values and speed up the process in the following cell
min_lat_bxl = interventions_bxl_coords['latitude_permanence'].min()
min_long_bxl = interventions_bxl_coords['longitude_permanence'].min()

min_lat_bxl2 = interventions_bxl_coords['latitude_permanence'].min()
min_long_bxl2 = interventions_bxl_coords['longitude_permanence'].min()

print(min_lat_bxl)
print(min_long_bxl)
print(min_lat_bxl2)
print(min_long_bxl2)

#so for latitude the initial divide can be by 10**4, for longitude it can be 10**3, based on the printed values:
#508047
#43089
#508047
#43089



508047
43089
508047
43089


'\n508047\n43089\n508047\n43089\n'

In [190]:
#adjusting the values for the bxl table, so that they are correctly formatted (with decimal point at the right place)
#latitude needs to be 2 numbers before the decimal, longitude 1 number before the decimal. We know that the entirety of Belgium lies  around rougly
#longitude 50.5 and latitude 4, so values deviating strongly from those (more than 2 units) are known to be invalid.

# fast solution to convert it, simply by dividing the values (using a simple while loop and iteratively dividing by 10 was too slow)
import math
def format_latitude(value):
    value / 10**4   #DIT KAN IK EIGENLIJK ERVOOR DOEN, AANGEZIEN HET HETZELFDE IS VOOR BEIDE KOLOMMEN, AL ERVOOR DELEN.
    if value > 99:
        value = value / (10**math.floor(math.log10(value) - 1))
    return value

# longitude needs to be 1 number before the decimal
def format_longitude(value):
    value / 10**4
    if value > 9:
        value = value / (10**math.floor(math.log10(value)))
    return value

interventions_bxl_coords['latitude_permanence'] = interventions_bxl_coords['latitude_permanence'].apply(format_latitude)
interventions_bxl_coords['longitude_permanence'] = interventions_bxl_coords['longitude_permanence'].apply(format_longitude)

interventions_bxl2_coords['latitude_permanence'] = interventions_bxl2_coords['latitude_permanence'].apply(format_latitude)
interventions_bxl2_coords['longitude_permanence'] = interventions_bxl2_coords['longitude_permanence'].apply(format_longitude)


In [204]:
# the values seem withing range, realistic coordinates.
testbxl = interventions_bxl_coords.value_counts()
testbxl2 = interventions_bxl2_coords.value_counts()

print(testbxl)
print(testbxl2)

latitude_permanence  longitude_permanence
50.859460            4.351810                18934
50.832540            4.311990                12996
50.834330            4.345450                11607
50.869480            4.386490                 9979
50.818870            4.403110                 9037
50.850970            4.364110                 8983
50.890870            4.308670                 7048
50.852110            4.460400                 4172
50.815980            4.341524                 3072
50.804700            4.367630                 3010
50.835710            4.347930                 2701
50.842260            4.399250                 2608
50.884920            4.308900                 2470
50.837580            4.349150                 2400
50.869370            4.386460                 2230
50.825730            4.379190                 2020
50.804690            4.367640                 1892
50.783440            4.356330                 1772
50.852300            4.359880           

In [206]:
# concatenating it into one big table
all_permanence_locations = pd.concat([interventions_bxl_coords,
                                     interventions_bxl2_coords,
                                     interventions1_coords,
                                     interventions2_coords,
                                     interventions3_coords,
                                     cad9_coords], ignore_index = True)
all_permanence_locations_unique = all_permanence_locations.drop_duplicates()

In [208]:
permanence_value_counts = all_permanence_locations.value_counts()
print(permanence_value_counts)

latitude_permanence  longitude_permanence
50.85946             4.35181                 18934
50.83254             4.31199                 12996
50.44608             3.91916                 11742
50.83433             4.34545                 11607
50.39143             4.42896                 11192
                                             ...  
50.30147             4.64535                     1
51.24501             4.48938                     1
50.31327             4.90912                     1
50.36155             3.49716                     1
49.98963             4.71100                     1
Name: count, Length: 654, dtype: int64


In [207]:
#check if the number of numbers after the decimal is equal for all values in the table, since that also influences the count of
#unique values, which might explain why there are more values here than in the hospital location table.

# check if the number of numbers after the decimal point is equal for all values

def count_decimal_places(x):
    s = str(x)
    if '.' in s:
        return len(s.split('.')[1])
    else:
        return 0

# apply the function to both columns
decimal_places_latitude = all_permanence_locations['latitude_permanence'].apply(count_decimal_places)
decimal_places_longitude = all_permanence_locations['longitude_permanence'].apply(count_decimal_places)

# combine the results and find the min and max
all_decimal_places = pd.concat([decimal_places_latitude, decimal_places_longitude])
min_decimal_places = all_decimal_places.min()
max_decimal_places = all_decimal_places.max()

# check if all values have the same number of decimal places
same_decimal_places = decimal_places_latitude.equals(decimal_places_longitude)

# Display results
print(f"Minimum number of decimal places: {min_decimal_places}")
print(f"Maximum number of decimal places: {max_decimal_places}")
print(f"All values have the same number of decimal places: {same_decimal_places}")

Minimum number of decimal places: 0
Maximum number of decimal places: 14
All values have the same number of decimal places: False


In [219]:
#the amount of decimals is all over the place, which is very problematic.

# get the count of each unique number of decimal places
decimal_places_count = all_decimal_places.value_counts().sort_index()

print("Count of each number of decimal places:")
print(decimal_places_count)

Count of each number of decimal places:
0      258768
3       21589
4       73640
5     1068696
6        1761
7       68219
8      226239
9      220842
10      19957
12       6627
13      45210
14      51807
15       7817
16      19926
Name: count, dtype: int64


In [210]:
# check if the amount of numbers before the decimal is 1 for longitude, and 2 for latitude, as they should be. This could be 
#part of the cause of the differing number of numbers.

def check_digits_before_decimal(x, expected_digits):
    s = str(x)
    if '.' in s:
        digits_before_decimal = len(s.split('.')[0])
        return digits_before_decimal == expected_digits
    else:
        return False

# apply the function to latitude (expecting 2 digits before decimal)
latitude_check = all_permanence_locations['latitude_permanence'].apply(check_digits_before_decimal, expected_digits=2)

# apply the function to longitude (expecting 1 digit before decimal)
longitude_check = all_permanence_locations['longitude_permanence'].apply(check_digits_before_decimal, expected_digits=1)

# check if all values meet the criteria
all_latitude_valid = latitude_check.all()
all_longitude_valid = longitude_check.all()

print(f"All 'latitude_permanence' values have 2 digits before the decimal point: {all_latitude_valid}")
print(f"All 'longitude_permanence' values have 1 digit before the decimal point: {all_longitude_valid}")

All 'latitude_permanence' values have 2 digits before the decimal point: False
All 'longitude_permanence' values have 1 digit before the decimal point: False


In [220]:
#we can simply apply the same function we used on the bxl coordinates again to all the data.

all_permanence_locations_formatted = all_permanence_locations
all_permanence_locations_formatted['latitude_permanence'] = all_permanence_locations_formatted['latitude_permanence'].apply(format_latitude)
all_permanence_locations_formatted['longitude_permanence'] = all_permanence_locations_formatted['longitude_permanence'].apply(format_longitude)

#there are 20 values less here, this implies that some coordinates were double coded but with different levels of precision (less decimals)
print(all_permanence_locations_formatted.value_counts())

latitude_permanence  longitude_permanence
50.85946             4.35181                 18934
50.83254             4.31199                 12996
50.44608             3.91916                 11742
50.83433             4.34545                 11607
50.39143             4.42896                 11192
                                             ...  
50.30147             4.64535                     1
50.36155             3.49716                     1
50.31327             4.90912                     1
51.20478             4.77508                     1
49.98963             4.71100                     1
Name: count, Length: 634, dtype: int64


In [221]:
#check again if the number of decimals is the same, to see if this solved the problem.

decimal_places_latitude = all_permanence_locations_formatted['latitude_permanence'].apply(count_decimal_places)
decimal_places_longitude = all_permanence_locations_formatted['longitude_permanence'].apply(count_decimal_places)

all_decimal_places = pd.concat([decimal_places_latitude, decimal_places_longitude])

min_decimal_places = all_decimal_places.min()
max_decimal_places = all_decimal_places.max()

# get the count of each unique number of decimal places
decimal_places_count = all_decimal_places.value_counts().sort_index()

# display results
print(f"Minimum number of decimal places: {min_decimal_places}")
print(f"Maximum number of decimal places: {max_decimal_places}")
print("Count of each number of decimal places:")
print(decimal_places_count)

Minimum number of decimal places: 0
Maximum number of decimal places: 16
Count of each number of decimal places:
0      258768
3       21589
4       73640
5     1068696
6        1761
7       68219
8      226239
9      220842
10      19957
12       6627
13      45210
14      51807
15       7817
16      19926
Name: count, dtype: int64


In [215]:
#the count of unique values is influenced by the number of numbers after the decimal point, e.g. 51.1234 is not the same as
# 51.123, with one less number after the decimal. Therefore the varying number of digits after the decimal point is a problem.
all_permanence_locations_unique_test = all_permanence_locations_formatted.drop_duplicates()
print(all_permanence_locations_unique_test)

         latitude_permanence  longitude_permanence
0                  50.850970              4.364110
2                  50.834330              4.345450
4                  50.852110              4.460400
6                  50.842260              4.399250
8                  50.869480              4.386490
...                      ...                   ...
952153             51.179015              4.361991
952385             51.156827              4.415969
978425             51.205074              4.396444
1030254            51.210317              4.051975
1036343            51.251317              4.248720

[925 rows x 2 columns]


In [222]:
#we cannot use the coordinates with fewer than 6 digits after the decimal point. It is possible however that all the coordinates
#with fewer than 6 digits have zeroes instead of other numbers, so for example 4.50000, where the zeroes have been removed. However, this is 
#extremely unlikely to be the case for all those cases.
#when removing all the coordinates with fewer than 6 numbers after the decimal, we end up with 226 unique values. This is fewer
#than the amount of coordinates from the hospital tables. Therefore, we will use the coordinates from the hospital table in all further analyses
#regarding the permanences.
#this again shows how inconsistent the data in general is.

#count and store the number of decimal places to use it to filter
decimal_places_latitude = all_permanence_locations_formatted['latitude_permanence'].apply(count_decimal_places)
decimal_places_longitude = all_permanence_locations_formatted['longitude_permanence'].apply(count_decimal_places)

# filter the DataFrame to keep rows where both columns have at least 6 decimal places
decimal_filtered_permanence_test = all_permanence_locations_formatted[(decimal_places_latitude >= 6) & (decimal_places_longitude >= 6)]


unique = decimal_filtered_permanence_test.drop_duplicates()
print(unique)

         latitude_permanence  longitude_permanence
2                  50.834330              4.345450
15                 50.825831              4.378717
26                 50.815980              4.341524
215                50.906541              4.388562
115647             50.842255              4.399238
...                      ...                   ...
952153             51.179015              4.361991
952385             51.156827              4.415969
978425             51.205074              4.396444
1030254            51.210317              4.051975
1036343            51.251317              4.248720

[226 rows x 2 columns]


In [232]:
#rename to make the merging of the tables possible.

#OOK DE COORDS NOG NODIG, DUS DAT KAN IK OOK AL DIRECT DOEN. EERST FILTEREN OP DE HARTSTILSTANDEN, dan is het sneller om mee te werken.
#tijden ook bijhouden
#of het afgebroken wordt, of er effectief wordt opgehaald, ook bijhouden, om te filteren.
#
cad9 = cad9.rename(columns={'EventType Trip': 'eventType_trip'})
interventions_bxl = interventions_bxl.rename(columns={'eventtype_trip': 'eventType_trip'})
interventions_bxl2 = interventions_bxl2.rename(columns={'EventType and EventLevel': 'eventType_trip'})
interventions1 = interventions1.rename(columns={'EventType Trip': 'eventType_trip'})
interventions2 = interventions2.rename(columns={'EventType Trip': 'eventType_trip'})
interventions3 = interventions3.rename(columns={'EventType Trip': 'eventType_trip'})

#drop columns not needed, makes it easier to work with the data
cad9_dropped.loc[:,['']]



In [231]:
#By checking the "Belgian manual for medical regulation", we found that the following codes are relevant to our research:
#P039 - CARDIAAL PROBLEEM (NIET PIJN OP DE BORST)
#P011 - PIJN OP DE BORST
#P003 - HARTSTILSTAND - DOOD - OVERLEDEN
#P008 - PATIËNT MET DEFIBRILLATOR OF PACEMAKER
#mainly 003 is relevant, but the other codes mentioned can also play a role in deciding where to place an AED.

In [235]:
#inspecting the variables to see which of them we should keep for further analysis

cad9_vars = cad9.columns.tolist()
interventions_bxl_vars = interventions_bxl.columns.tolist()
interventions_bxl2_vars = interventions_bxl2.columns.tolist()
interventions1_vars = interventions1.columns.tolist()
interventions2_vars = interventions2.columns.tolist()
interventions3_vars = interventions3.columns.tolist()

print(cad9_vars)
'eventType_trip'
'T0'
'T3'

print(interventions_bxl_vars)
'eventType_trip'
't0'
't3'
'number_of_transported_persons'
'abandon_reason'

print(interventions_bxl2_vars)
'T0'
'eventType_trip'
'Number of transported persons'
'Abandon reason NL', 'Abandon reason FR'
'T3'

print(interventions1_vars)
'eventType_trip'
'T0'
'T3'
'Number of transported persons'
'Abandon reason'

print(interventions2_vars)
'eventType_trip'
'T0'
'T3'
'Number of transported persons'
'Abandon reason'

print(interventions3_vars)
'eventType_trip'
'T0'
'T3'
'Number of transported persons'
'Abandon reason'

['province', 'Mission ID', 'Service Name', 'Latitude permanence', 'Longitude permanence', 'Permanence short name', 'Permanence long name', 'Vector Type', 'eventType_trip', 'EventSubType Trip', 'EventLevel Trip', 'CityName intervention', 'CitysectionName intervention', 'Latitude intervention', 'Longitude intervention', 'Province invervention', 'T0', 'T1', 'T1confirmed', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'Name destination hospital', 'Intervention time (T1Reported)', 'Intervention time (T1Confirmed)', 'Departure time (T1Reported)', 'Departure time (T1Confirmed)', 'UI', 'ID', 'MISSION_NR', 'AMBUCODE', 'UNIT_ID']
['mission_id', 'service_name', 'postalcode_permanence', 'cityname_permanence', 'streetname_permanence', 'housenumber_permanence', 'latitude_permanence', 'longitude_permanence', 'permanence_short_name', 'permanence_long_name', 'vector_type', 'eventtype_firstcall', 'eventLevel_firstcall', 'eventType_trip', 'eventlevel_trip', 'postalcode_intervention', 'cityname_intervention', 'lati

In [241]:
#renaming the other variables too

interventions_bxl = interventions_bxl.rename(columns={'t0': 'T0',
                                                     't3': 'T3'})
interventions_bxl2 = interventions_bxl2.rename(columns={'Number of transported persons': 'number_of_transported_persons',
                                                       'Abandon reason NL': 'abandon_reason'})
interventions1 = interventions1.rename(columns={'Number of transported persons': 'number_of_transported_persons',
                                                       'Abandon reason': 'abandon_reason'})
interventions2 = interventions2.rename(columns={'Number of transported persons': 'number_of_transported_persons',
                                                       'Abandon reason': 'abandon_reason'})
interventions3 = interventions3.rename(columns={'Number of transported persons': 'number_of_transported_persons',
                                                       'Abandon reason': 'abandon_reason'})

In [245]:
common_variables = ['eventType_trip', 'T0', 'T3', 'number_of_transported_persons', 'abandon_reason']
interventions_bxl_filtered = interventions_bxl.filter(common_variables)
interventions_bxl2_filtered = interventions_bxl2.filter(common_variables)
interventions1_filtered = interventions1.filter(common_variables)
interventions2_filtered = interventions2.filter(common_variables)
interventions3_filtered = interventions3.filter(common_variables)
cad9_filtered = cad9.filter(['eventType_trip', 'T0', 'T3'])


In [246]:
#nu nog de locaties van de harstilstanden. Eerst filteren op hartstilstand, dan de locaties eruit halen.
card_arrest = pd.concat([interventions_bxl_filtered,
                        interventions_bxl2_filtered,
                        interventions1_filtered,
                        interventions2_filtered,
                        interventions3_filtered,
                        cad9_filtered
                        ],
                        ignore_index = True)
card_arrest
#evnttype is de var waar het staat, nu nog de code vinden voor de hartstilstanden.
#eventtype trip is used, since it provides the latest information on the reason for the call

,eventType_trip,T0,T3,number_of_transported_persons,abandon_reason
0,P033 - Trauma,2022-09-06 11:49:21.5868598 +02:00,None,NaN,Error
1,P033 - Trauma,2022-09-06 11:49:21.5868598 +02:00,2022-09-06 10:07:00.7842800 +00:00,NaN,None
2,P059 - Dizziness - Nausea,2022-09-06 11:55:35.7936791 +02:00,None,NaN,Error
3,P019 - Unconscious - syncope,2022-09-06 12:39:23.4337324 +02:00,None,NaN,Weigering van vervoer
4,P033 - Trauma,2022-09-06 13:26:48.3379147 +02:00,None,NaN,Geannuleerd
...,...,...,...,...,...
1045544,PERSONNES en danger / dans le besoin,None,2022-07-06 18:40:52.000,NaN,NaN
1045545,PERSONNES en danger / dans le besoin,None,2022-07-06 19:08:28.000,NaN,NaN
1045546,PERSONNES en danger / dans le besoin,None,2022-07-06 19:20:02.000,NaN,NaN
1045547,PERSONNES en danger / dans le besoin,None,2022-07-07 12:57:39.000,NaN,NaN
